In [1]:
import pandas as pd
import re

In [2]:
"""
Test parameters

input: 64 engineered features extracted from lidc max-slice images

model layer sizes = (64, 32, 32, 2)
groupdro_eta = 0.1
lr = 0.001
weight_decay = 0.005

batch_size = 40
proportional = True
epochs = 40
N = 120

"""

'\nTest parameters\n\ninput: 64 engineered features extracted from lidc max-slice images\n\nmodel layer sizes = (64, 32, 32, 2)\ngroupdro_eta = 0.1\nlr = 0.001\nweight_decay = 0.005\n\nbatch_size = 40\nproportional = True\nepochs = 40\nN = 120\n\n'

In [3]:
results = pd.read_csv("../results").iloc[:, 1:].transpose()
results.columns = ["ERM", "GDRO"]

In [4]:
results

,ERM,GDRO
0,"(0.8585526315789473, array([0.875 , 0.8605...","(0.7861842105263158, array([0.84375 , 0.7740..."
1,"(0.8651315789473685, array([0.9375 , 0.8461...","(0.8092105263157895, array([0.875 , 0.8076..."
2,"(0.8453947368421053, array([0.9375, 0.8125, 0....","(0.8157894736842105, array([0.890625 , 0.8076..."
3,"(0.8289473684210527, array([0.96875, 0.8125 , ...","(0.8552631578947368, array([0.890625 , 0.8509..."
4,"(0.8355263157894737, array([0.890625 , 0.8413...","(0.8782894736842105, array([0.9375 , 0.8509..."
...,...,...
115,"(0.8125, array([0.890625 , 0.81730769, 0.8125...","(0.8421052631578947, array([0.9375 , 0.8365..."
116,"(0.8486842105263158, array([0.921875 , 0.8317...","(0.8223684210526315, array([0.90625 , 0.8365..."
117,"(0.8717105263157895, array([0.953125 , 0.8605...","(0.875, array([0.890625 , 0.86538462, 1. ..."
118,"(0.8618421052631579, array([0.875 , 0.8605...","(0.8289473684210527, array([0.890625 , 0.8173..."


In [7]:
split = [[[float(s) for s in re.sub("\(|\[|[a-z]| |\]|\)", "", v).split(",")] for v in row] for row in results.values]
split[0][0][0]

0.8585526315789473

In [8]:
data_clean = [pd.DataFrame([[val[group] for val in row] for row in split], columns=["ERM","GDRO"]) for group in range(5)]

In [13]:
from scipy.stats import ttest_ind
import statistics

group_names = ["Overall", "GE MEDICAL SYSTEMS", "Unknown", "SIEMENS", "TOSHIBA", "Philips"]
# group_names = ["Overall", "unmarked_benign", "marked_benign", "marked_malignant", "unmarked_malignant"]
# group_names = ["Overall", "0benign", "1benign", "0malignant", "1malignant"]
for i in range(5):
    print(group_names[i])
    print(f"Mean ERM accuracy: {statistics.mean(data_clean[i]['ERM'])}")
    print(f"Mean GDRO accuracy: {statistics.mean(data_clean[i]['GDRO'])}")
    stat, p = ttest_ind(data_clean[i]["ERM"], data_clean[i]["GDRO"])
    if p < 0.05:
        print(f"{'ERM' if stat > 0 else 'GDRO'} higher accuracy (p = {p})")
    else:
        print(f"No statistically significant difference (p = {p})")
    print()
    

Overall
Mean ERM accuracy: 0.8439692982456141
Mean GDRO accuracy: 0.8448739035087719
No statistically significant difference (p = 0.7486879873250809)

GE MEDICAL SYSTEMS
Mean ERM accuracy: 0.90234375
Mean GDRO accuracy: 0.9045572916666667
No statistically significant difference (p = 0.683852934869311)

Unknown
Mean ERM accuracy: 0.84118589725
Mean GDRO accuracy: 0.8420272439166667
No statistically significant difference (p = 0.8037516811744757)

SIEMENS
Mean ERM accuracy: 0.8354166666666667
Mean GDRO accuracy: 0.825
No statistically significant difference (p = 0.4173936799349669)

TOSHIBA
Mean ERM accuracy: 0.6020833333333333
Mean GDRO accuracy: 0.56875
No statistically significant difference (p = 0.21869716894896418)



In [31]:
# old results with flawed malignancy labels + CNN subtypes
data_clean0

[          ERM      GDRO
 0    0.809211  0.812500
 1    0.822368  0.812500
 2    0.792763  0.812500
 3    0.792763  0.812500
 4    0.786184  0.809211
 ..        ...       ...
 115  0.812500  0.825658
 116  0.812500  0.786184
 117  0.809211  0.851974
 118  0.812500  0.786184
 119  0.805921  0.835526
 
 [120 rows x 2 columns],
           ERM      GDRO
 0    0.843750  0.864583
 1    0.833333  0.916667
 2    0.833333  0.770833
 3    0.968750  0.843750
 4    0.833333  0.760417
 ..        ...       ...
 115  0.895833  0.843750
 116  0.906250  0.791667
 117  0.916667  0.947917
 118  0.802083  0.864583
 119  0.864583  0.885417
 
 [120 rows x 2 columns],
           ERM      GDRO
 0    0.895833  0.864583
 1    0.854167  0.885417
 2    0.854167  0.864583
 3    0.875000  0.916667
 4    0.927083  0.854167
 ..        ...       ...
 115  0.875000  0.864583
 116  0.937500  0.791667
 117  0.875000  0.802083
 118  0.875000  0.781250
 119  0.906250  0.916667
 
 [120 rows x 2 columns],
          ERM     G